In [1]:
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import torchvision
import torch
import os
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve, auc
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
import pickle
from sklearn.metrics import accuracy_score, f1_score
import torch.utils.data as data_utils
from collections import defaultdict
import gc

In [ ]:
# utils for cluster generation
class Dataset:
    def __init__(self, file='data/icons_embd.p', train=True, classes=[]):

        self.data = pickle.load(open(file, 'rb'))[0] 
        self.targets = pickle.load(open(file, 'rb'))[1]       
        self.classes = classes

In [ ]:
seeds = [1234,6265, 2403, 941,2225]
label_noises = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#please, change accordingly to the machine
n_cores = 30

In [ ]:
###generate clusters using not permutted(not noisy) data 
for run, seed in enumerate(seeds):
    print("Generating clusters...")
    file = pickle.load(open('../../../teamdrive/transmediasp/kate/hierarchy/icons_embd_%s.p' %run, 'rb'))
    trainset = Dataset(file)
    n_props = 1

    generate_dist_matrix_kwargs = {'metric':'tasksim', 
                                   'metric_kwargs':{'n_neg_classes': 10, 
                                                    'task_similarity_kwargs': {'transformer_kwargsx': 
                                                                                   {'max_depth':4},
                                                                              'transformer_kwargsz':
                                                                                  {'max_depth':4}}}, 
                                   'function_tuples':None, 
                                   'n_cores':n_cores, 
                                   'acorn':seed
                                  }


    process_dist_matrix_kwargs = {'make_symmetric': True,
                                  'scale':True,
                                 'aug_diag':True,
                                 }

    embedding=ASE
    embedding_kwargs={'n_components':16}
    cluster=GMM
    cluster_kwargs = {'min_components': 20, 'max_components': 20, 'reg_covar': 1e-03}

    cluster_dists_kwargs = {'embedding':embedding, 
                            'embedding_kwargs':embedding_kwargs, 
                            'cluster':cluster, 
                            'cluster_kwargs':cluster_kwargs
                           }

    n_mc=1
    master_seed = seed
    np.random.seed(master_seed)

    data_dimension=2048

    generate_tasksim=True

    if generate_tasksim:
        tasksim_clusters = []

    #- Generate clusters
    for iteration in tqdm(range(n_mc)):
        start = time.time()
        X_train, _, y_train, _ = train_test_split(trainset.data, trainset.targets, test_size=0.1, random_state=seed)
        print(len(X_train), len(y_train))
        if generate_tasksim:
            temp_tasksim = generate_hierarchy(X_train, y_train, generate_dist_matrix_kwargs, process_dist_matrix_kwargs, cluster_dists_kwargs)
            tasksim_clusters.append(temp_tasksim)

            pickle.dump(tasksim_clusters, open('../../../teamdrive/transmediasp/kate/hierarchy/icons_tasksim_clusters_%s.p' %run, 'wb'))
    print("Clusters are generated for run %s" %run)   


In [ ]:
###generate clusters using permutted(noisy) data 

df = pd.read_csv('../../../teamdrive/transmediasp/kate/icons_df.csv')

for run, seed in enumerate(seeds):
    for label_noise in label_noises:
        label_str = str(int(label_noise*100))
        
        print("Generating clusters...")
        file = pickle.load(open('../../../teamdrive/transmediasp/kate/hierarchy/icons_embd_%s.p' %run, 'rb'))
        trainset = Dataset(file)
        trainset.targets = np.array(df[df['run_%s' %run] == 'main']['label_%s_%s' %(run, label_str)])
        n_props = 1

        generate_dist_matrix_kwargs = {'metric':'tasksim', 
                                       'metric_kwargs':{'n_neg_classes': 10, 
                                                        'task_similarity_kwargs': {'transformer_kwargsx': 
                                                                                       {'max_depth':4},
                                                                                  'transformer_kwargsz':
                                                                                      {'max_depth':4}}}, 
                                       'function_tuples':None, 
                                       'n_cores':n_cores, 
                                       'acorn':seed
                                      }


        process_dist_matrix_kwargs = {'make_symmetric': True,
                                      'scale':True,
                                     'aug_diag':True,
                                     }

        embedding=ASE
        embedding_kwargs={'n_components':16}
        cluster=GMM
        cluster_kwargs = {'min_components': 20, 'max_components': 20, 'reg_covar': 1e-03}

        cluster_dists_kwargs = {'embedding':embedding, 
                                'embedding_kwargs':embedding_kwargs, 
                                'cluster':cluster, 
                                'cluster_kwargs':cluster_kwargs
                               }

        n_mc=1
        master_seed = seed
        np.random.seed(master_seed)

        data_dimension=2048

        generate_tasksim=True

        if generate_tasksim:
            tasksim_clusters = []

        #- Generate clusters
        for iteration in tqdm(range(n_mc)):
            start = time.time()
            X_train, _, y_train, _ = train_test_split(trainset.data, trainset.targets, test_size=0.1, random_state=seed)
            print(len(X_train), len(y_train))
            if generate_tasksim:
                temp_tasksim = generate_hierarchy(X_train, y_train, generate_dist_matrix_kwargs, process_dist_matrix_kwargs, cluster_dists_kwargs)
                tasksim_clusters.append(temp_tasksim)

                pickle.dump(tasksim_clusters, open('../../../teamdrive/transmediasp/kate/hierarchy/icons_tasksim_clusters_%s_%s.p' %(run, label_str), 'wb'))
        print("Clusters are generated for run %s, label noise %s" %(run, label_noise))   
